In [ ]:
import re, json, pickle
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

In [ ]:
good_questions = np.load('Data/OpinionQA/good_questions.npy') # question_id's to use
Qs = pd.read_csv('Data/OpinionQA/Qs_to_use.csv') # for the good questions, contains question statement and choices
with open('Data/OpinionQA/surveys.pkl', 'rb') as f:
    surveys = pickle.load(f) # key: question id, value: survey samples
with open('Data/OpinionQA/synthetic_profiles.pkl', 'rb') as f:
    synthetic_profiles = pickle.load(f) # key: question id, value: synthetic profiles
with open('Data/OpinionQA/choices_to_numeric.json', 'r') as f:
    choices_to_numeric = json.load(f) # key: question id, value: choices to numeric mapping

# calculate the similarity between two strings
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
import openai
api_key = '' # insert OpenAI API key here
client = openai.OpenAI(api_key = api_key)

In [ ]:
# example of using the OpenAI API to simulate an answer
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are simulating the behaviors of humans with certain specified characteristics to help with a survey study."},
        {
            "role": "user",
            "content": synthetic_profiles[good_questions[0]]['PROMPT'].values[1]
        }
    ]
)

print(completion.choices[0].message.content)

[[2]]


In [ ]:
# call api to obtain simulated answers
synthetic_answers_raw = {}
num_of_synthetic_answers = 100
for i in range(len(good_questions)):
    synthetic_answers_raw[good_questions[i]] = []
    for j in range(num_of_synthetic_answers):
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are simulating the behaviors of humans with certain specified characteristics to help with a survey study."},
                {
                    "role": "user",
                    "content": synthetic_profiles[good_questions[i]]['PROMPT'].values[j]
                }
            ],
            seed = 1
        )
        synthetic_answers_raw[good_questions[i]].append(completion.choices[0].message.content)
        print('Question ' + str(i) + ' Answer ' + str(j) + ' Done', end = '\r')
    
    with open('Data/OpinionQA/synthetic answers/raw/synthetic_answers_raw_4o.pkl', 'wb') as f:
        pickle.dump(synthetic_answers_raw, f)

In [ ]:
# find all the answers
synthetic_answers = {}
for i in range(len(good_questions)):
    synthetic_answers[good_questions[i]] = []
    for j in range(num_of_synthetic_answers):
        # use the double brackets to indicate the answer
        answer = re.findall(r'\[\[(.*?)\]', synthetic_answers_raw[good_questions[i]][j])
        answer = answer[0]
        # extract the choice number, if there is any
        answer_num = re.findall(r'\d+', answer)
        # if there is no choice number, use the similarity to find the most similar choice
        if len(answer_num) == 0:
            ans_to_num_i = choices_to_numeric[good_questions[i]]
            sims = [similarity(answer, key) for key in ans_to_num_i.keys()]
            max_sim_ind = np.argmax(sims)
            answer_num = [1, 2, 3, 4, 5][max_sim_ind]
        else:
            answer_num = answer_num[0]
        synthetic_answers[good_questions[i]].append(answer_num)

In [ ]:
# convert the answers to numeric
synthetic_answers_numeric = {}
num_to_num = {'1': 1, '2': 1/3, '3': -1/3, '4': -1, '5': 0}
for i in range(len(good_questions)):
    numeric_i = []
    for j in range(num_of_synthetic_answers):
        numeric_i.append(num_to_num[str(synthetic_answers[good_questions[i]][j])])
    synthetic_answers_numeric[good_questions[i]] = numeric_i

In [ ]:
with open('Data/OpinionQA/synthetic answers/clean/synthetic_answers_clean_4o.pkl', 'wb') as f:
    pickle.dump(synthetic_answers_numeric, f)